In [9]:
import pandas as pd

filmes = pd.read_csv("movies.csv")
filmes.columns = ["filmeId", "titulo", "generos"]
filmes = filmes.set_index("filmeId")
filmes.head()

,titulo,generos
filmeId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [4]:
notas = pd.read_csv("ratings.csv")
notas.columns = ["usuarioId", "filmeId", "nota", "momento"]
notas.head()

,usuarioId,filmeId,nota,momento
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [5]:
notas.describe()

,usuarioId,filmeId,nota,momento
count,492743.000000,492743.000000,492743.000000,4.927430e+05
mean,2481.021307,19148.468963,3.521933,1.191202e+09
std,1453.661262,36210.993514,1.062460,2.159963e+08
min,1.000000,1.000000,0.500000,8.256384e+08
25%,1200.000000,1090.000000,3.000000,1.001242e+09
50%,2428.000000,2719.000000,3.500000,1.171580e+09
75%,3791.000000,7215.500000,4.000000,1.418069e+09
max,4998.000000,193861.000000,5.000000,1.537927e+09


# Primeira tentativa de recomendação: heurística de total de votos

In [11]:
total_de_votos = notas["filmeId"].value_counts()
total_de_votos.head()

318     1739
356     1698
296     1628
593     1564
2571    1437
Name: filmeId, dtype: int64

In [12]:
filmes['total_de_votos'] = total_de_votos
filmes.head()

,titulo,generos,total_de_votos
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1212.0
2,Jumanji (1995),Adventure|Children|Fantasy,467.0
3,Grumpier Old Men (1995),Comedy|Romance,287.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,62.0
5,Father of the Bride Part II (1995),Comedy,275.0


In [16]:
filmes.sort_values("total_de_votos", ascending = False).head(10)

,titulo,generos,total_de_votos
filmeId,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,1739.0
356,Forrest Gump (1994),Comedy|Drama|Romance|War,1698.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1628.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,1564.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1437.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1421.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,1322.0
527,Schindler's List (1993),Drama|War,1299.0
110,Braveheart (1995),Action|Drama|War,1269.0


In [21]:
notas_medias = notas.groupby("filmeId").mean()["nota"]
notas_medias.head()

filmeId
1    3.888614
2    3.140257
3    3.175958
4    2.943548
5    3.016364
Name: nota, dtype: float64

In [23]:
filmes["nota_media"] = notas_medias
filmes.sort_values("total_de_votos", ascending = False).head(10)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,1739.0,4.410293
356,Forrest Gump (1994),Comedy|Drama|Romance|War,1698.0,4.087161
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1628.0,4.174140
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,1564.0,4.149936
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,1437.0,4.124913
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1421.0,4.087262
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,1322.0,3.656203
527,Schindler's List (1993),Drama|War,1299.0,4.265204
110,Braveheart (1995),Action|Drama|War,1269.0,4.008668


# Uma segunda heurística: nota média e filtrando votos

In [24]:
filmes.sort_values("nota_media", ascending = False).head(10)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
76171,India (Indien) (1993),Comedy|Drama,1.0,5.0
115943,The Land Before Time V: The Mysterious Island ...,Adventure|Animation|Children,1.0,5.0
117368,The Madagascar Penguins in a Christmas Caper (...,Animation|Comedy,1.0,5.0
117310,Aashiqui 2 (2013),Drama|Musical|Romance,1.0,5.0
158486,Side Effects (2005),Comedy|Drama|Romance,1.0,5.0
50942,"Wake Up, Ron Burgundy (2004)",Comedy,1.0,5.0
27235,"Shrink Is In, The (2001)",Comedy|Romance,1.0,5.0
158878,The Letter Writer (2011),Children,1.0,5.0
116493,The Returned (2013),Drama|Horror|Thriller,1.0,5.0


In [25]:
filmes.query("total_de_votos >= 10").sort_values("nota_media", ascending = False).head(10)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
69849,Roots (1977),Drama|War,10.0,4.450000
174053,Black Mirror: White Christmas (2014),Drama|Horror|Mystery|Sci-Fi|Thriller,26.0,4.423077
7396,Scenes From a Marriage (Scener ur ett äktenska...,Drama,12.0,4.416667
318,"Shawshank Redemption, The (1994)",Crime|Drama,1739.0,4.410293
6650,Kind Hearts and Coronets (1949),Comedy|Drama,18.0,4.361111
6852,In Cold Blood (1967),Crime|Drama,14.0,4.357143
163134,Your Name. (2016),Animation|Drama|Fantasy|Romance,24.0,4.354167
55908,"Man from Earth, The (2007)",Drama|Sci-Fi,58.0,4.353448
5604,"Man in the White Suit, The (1951)",Comedy|Sci-Fi,10.0,4.350000


In [27]:
filmes_com_mais_de_50_votos = filmes.query("total_de_votos >= 50")
filmes_com_mais_de_50_votos.sort_values("nota_media", ascending = False).head(10)


,titulo,generos,total_de_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,1739.0,4.410293
55908,"Man from Earth, The (2007)",Drama|Sci-Fi,58.0,4.353448
858,"Godfather, The (1972)",Crime|Drama,1053.0,4.307217
50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1123.0,4.303206
678,Some Folks Call It a Sling Blade (1993),Drama|Thriller,51.0,4.294118
3359,Breaking Away (1979),Comedy|Drama,64.0,4.289062
926,All About Eve (1950),Drama,99.0,4.277778
527,Schindler's List (1993),Drama|War,1299.0,4.265204
1280,Raise the Red Lantern (Da hong deng long gao g...,Drama,57.0,4.263158


In [28]:
eu_assisti = [1, 21, 19, 10, 11, 7, 2]
filmes.loc[eu_assisti]

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1212.0,3.888614
21,Get Shorty (1995),Comedy|Crime|Thriller,460.0,3.528261
19,Ace Ventura: When Nature Calls (1995),Comedy,452.0,2.713496
10,GoldenEye (1995),Action|Adventure|Thriller,588.0,3.468537
11,"American President, The (1995)",Comedy|Drama|Romance,345.0,3.634783
7,Sabrina (1995),Comedy|Romance,276.0,3.304348
2,Jumanji (1995),Adventure|Children|Fantasy,467.0,3.140257


In [34]:
aventura_infantil_e_fantasia = filmes_com_mais_de_50_votos.query("generos=='Adventure|Children|Fantasy'")
aventura_infantil_e_fantasia.drop(eu_assisti, errors='ignore').sort_values("nota_media", ascending = False).head(10)

,titulo,generos,total_de_votos,nota_media
filmeId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,465.0,3.670968
2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,193.0,3.396373
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,221.0,3.391403
56171,"Golden Compass, The (2007)",Adventure|Children|Fantasy,50.0,3.170000
60,"Indian in the Cupboard, The (1995)",Adventure|Children|Fantasy,116.0,3.021552


In [0]:
# Collaborative filtering x Content based filtering